# install and Load packages

In [2]:
import pandas as pd
import numpy as np
import os
import time
from tqdm.auto import tqdm
import ee

In [ ]:
# mount the google drive
# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/master_thesis/

Mounted at /content/gdrive
/content/gdrive/MyDrive/master_thesis


In [13]:
from satimg_utils.gee_classes import *
from satimg_utils.gee_helpers import *
from satimg_utils.gee_exporters import *

# Download images

In [3]:
ee.Authenticate()

Enter verification code: 4/1Adeu5BUE55oxWl3dy9xwXW3PRmNTasl93CuWLumGknqgaW1H9hwqonsNogE

Successfully saved authorization token.


In [4]:
ee.Initialize()

In [18]:
# input data paths
root_data_dir = '../../Data'
LS_dir = f"{root_data_dir}/satellite_imgs/LS/LS_raw/"

LSMS_CSV_PATH = f'{root_data_dir}/lsms/processed/labels_cluster_v1.csv'

In [6]:
lsms_df = pd.read_csv(LSMS_CSV_PATH, float_precision='high')
lsms_df = lsms_df[['country', 'start_ts', 'lat', 'lon', 'series', 'cluster_id', 'unique_id']].copy()
lsms_df['LS_file_name'] = 'LS_'+lsms_df['unique_id'] + ".tif"

In [ ]:
# mask = [i in downloaded_files for i in lsms_df['file_name']]
# mask = ~np.array(mask)
# lsms_df = lsms_df.loc[mask,:].reset_index(drop = True)
# lsms_df = lsms_df[['country', 'start_ts', 'lat', 'lon', 'series', 'cluster_id']]
# if(len(lsms_df) == 0):
#   print("All data has been downloaded")

In [19]:
ls_downloaded_files = check_downloaded_files(prefix = "LS", source_dir = LS_dir)
ls_mask = ~np.array([i in ls_downloaded_files for i in lsms_df['LS_file_name']])

to_download_df = lsms_df.loc[ls_mask,:].copy().reset_index(drop = True)

In [20]:
to_download_df

,country,start_ts,lat,lon,series,cluster_id,unique_id,LS_file_name


In [21]:
# Download Landsat images
lsms_surveys = list(to_download_df.groupby(['country', 'series', 'start_ts', 'cluster_id', 'unique_id']).groups.keys())

tasks = {}
tab_feats = []
count = 0

for country, series, start_ts, cluster_id, unique_id in tqdm(lsms_surveys):
    new_tasks, feats = export_ls_images(
        df=lsms_df, country=country,
        series = series, start_ts = start_ts,
        unique_id = unique_id)
    tasks.update(new_tasks)
    tab_feats.append(feats)
    tab_feats_df = pd.DataFrame(tab_feats)
    if 'tabular_ls_features_v9.csv' in os.listdir("Data/LS/"):
      tab_feats_df.to_csv('Data/LS/tabular_ls_features_v9.csv', index = False, mode = 'a', header = False)
    else:
      tab_feats_df.to_csv('Data/LS/tabular_ls_features_v9.csv', index = False)
    count += 1
    if count == 2999:
      print(f'Going to sleep for {60*120} seconds')
      time.sleep(60*120)
      count = 0

0it [00:00, ?it/s]

In [22]:
wait_on_tasks(tasks)

0it [00:00, ?it/s]